In [ ]:
import pandas as pd
from models.models import Movie
from pymongo import MongoClient
import json

def make_recommendation(get_title):
    host = 'cluster0.xc44g.mongodb.net'
    user = 'aimb'
    password = 'mimomimo'
    database_name = 'frated'

    client = MongoClient(f"mongodb+srv://{user}:{password}@{host}/{database_name}?retryWrites=true&w=majority")    # db = client.frated
    db = client['frated']
    #select the collection within the database
    test = db.movie
    #convert entire collection to Pandas dataframe
    df = pd.DataFrame(list(test.find()))

    # df =  pd.DataFrame(iter(cursor))
    # movies = Movie.objects().to_json()
    #df =  pd.read_json('imdb_0328_key.json')
    #print(df)

    # 이걸 db에서 가져와야 하는데...
    df2 = df[['Title', 'Director', 'Actors', 'keywords', 'Genre', 'Awards']]

    # Actor, Genre 3개만 남겨 분석하는게 더 추천 정확도가 높을 수도...?

    df2['Actors'] = df['Actors'].str.split(',').str[:3]
    df2['Genre'] = df['Genre'].str.split(',').str[:3]

    def create_soup(x):
        # ''.join은 리스트 요소를 스트링으로 만들되 요소끼리 띄우지 않는다. 만약 ' '을 넣으면 알파벳끼리 떨어짐
        return ''.join(x['Genre'])+ ' ' + ''.join(x['Actors']) + ' ' +''.join(x['keywords']) +  ' ' + x['Director']  + ' ' + x['Awards']
    df2['soup'] = df2.apply(create_soup, axis=1)

    # Import CountVectorizer and create the count matrix
    from sklearn.feature_extraction.text import CountVectorizer

    count = CountVectorizer(stop_words='english')
    count_matrix = count.fit_transform(df2['soup'])

    # Compute the Cosine Similarity matrix based on the count_matrix
    from sklearn.metrics.pairwise import cosine_similarity

    cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

    # Reset index of our main DataFrame and construct reverse mapping as before
    df2 = df2.reset_index()
    indices = pd.Series(df2.index, index=df2['Title'])

    #Import TfIdfVectorizer from scikit-learn
    from sklearn.feature_extraction.text import TfidfVectorizer

    #Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
    tfidf = TfidfVectorizer(stop_words='english')

    #Construct the required TF-IDF matrix by fitting and transforming the data
    tfidf_matrix = tfidf.fit_transform(df2['soup'])

    #Output the shape of tfidf_matrix
    tfidf_matrix.shape

    # Import linear_kernel
    from sklearn.metrics.pairwise import linear_kernel

    # Compute the cosine similarity matrix
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

    # Function that takes in movie title as input and outputs most similar movies
    def get_recommendations(title, cosine_sim=cosine_sim):
        # Get the index of the movie that matches the title
        idx = indices[title]

        # Get the pairwsie similarity scores of all movies with that movie
        sim_scores = list(enumerate(cosine_sim[idx]))

        # Sort the movies based on the similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # Get the scores of the 5 most similar movies
        sim_scores = sim_scores[1:6] # 10개도 가능

        # Get the movie indices
        movie_indices = [i[0] for i in sim_scores]

        # Return the top 10 most similar movies
        return df2['Title'].iloc[movie_indices]

    get_title = get_title
    results = get_recommendations(get_title, cosine_sim2)
    return results